In [19]:
#Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time
from scipy.stats import linregress
from citipy import citipy
from config import weather_key

In [3]:
#Generating random latitude/longitudes to collect random cities
lat_range = (-90, 90)
lng_range = (-180, 180)

lat_lng = []
cities = []

lats = np.random.uniform(low=-90, high=90, size=1500)
lngs = np.random.uniform(low=-180, high=180, size=1500)
lat_lng = zip(lats, lngs)

for lat_lng in lat_lng:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    if city not in cities:
        cities.append(city)
        
len(cities)

592

In [18]:
#Performing API calls for each city
base_url = "https://api.openweathermap.org/data/2.5/weather?q="
record = 1

city_names = []
cloudiness = []
countries = []
dates = []
humidity = []
city_lats = []
city_lngs = []
max_temps = []
wind_speed = []

print("""Beginning Data Retrieval
------------------------------""")

for city in cities:
    try:
        target_url = f"{base_url}{city}&appid={weather_key}"
        response = requests.get(target_url)
        response_json = response.json()
        city_names.append(response_json['name'])
        cloudiness.append(response_json['clouds']['all'])
        countries.append(response_json['sys']['country'])
        dates.append(response_json['dt'])
        humidity.append(response_json['main']['humidity'])
        city_lats.append(response_json['coord']['lat'])
        city_lngs.append(response_json['coord']['lon'])
        max_temps.append(response_json['main']['temp_max'])
        wind_speed.append(response_json['wind']['speed'])
        print(f"Processing record {record} of {len(cities)}| {city}")
        record += 1
    except:
        print("City not found. Skipping...")
        record += 1
        
print("""------------------------------
Data Retrieval Complete
------------------------------""")

Beginning Data Retrieval
------------------------------
Processing record 1 of 592| albany
Processing record 2 of 592| castro
Processing record 3 of 592| camacha
Processing record 4 of 592| bluff
Processing record 5 of 592| kruisfontein
Processing record 6 of 592| taksimo
Processing record 7 of 592| quime
Processing record 8 of 592| coquimbo
Processing record 9 of 592| bedele
City not found. Skipping...
Processing record 11 of 592| nchelenge
Processing record 12 of 592| tuktoyaktuk
City not found. Skipping...
Processing record 14 of 592| jucuaran
Processing record 15 of 592| diamantino
Processing record 16 of 592| bada
Processing record 17 of 592| shkoder
Processing record 18 of 592| carnarvon
Processing record 19 of 592| port alfred
Processing record 20 of 592| esperance
Processing record 21 of 592| pithoragarh
Processing record 22 of 592| hilo
Processing record 23 of 592| sarahan
Processing record 24 of 592| mataura
Processing record 25 of 592| rikitea
Processing record 26 of 592| hu

Processing record 214 of 592| caraquet
Processing record 215 of 592| upernavik
Processing record 216 of 592| vostok
City not found. Skipping...
Processing record 218 of 592| qasigiannguit
Processing record 219 of 592| ballina
Processing record 220 of 592| ingham
Processing record 221 of 592| manokwari
Processing record 222 of 592| skjervoy
City not found. Skipping...
Processing record 224 of 592| obo
Processing record 225 of 592| port macquarie
Processing record 226 of 592| rawson
City not found. Skipping...
City not found. Skipping...
Processing record 229 of 592| grand gaube
Processing record 230 of 592| svetlaya
Processing record 231 of 592| aklavik
Processing record 232 of 592| grindavik
Processing record 233 of 592| bathsheba
Processing record 234 of 592| komsomolskiy
Processing record 235 of 592| qaanaaq
Processing record 236 of 592| tommot
Processing record 237 of 592| amahai
Processing record 238 of 592| mazamitla
Processing record 239 of 592| kidal
Processing record 240 of 592

City not found. Skipping...
Processing record 429 of 592| leninskoye
Processing record 430 of 592| chegdomyn
Processing record 431 of 592| mayumba
Processing record 432 of 592| vezirkopru
Processing record 433 of 592| clyde river
Processing record 434 of 592| tadine
Processing record 435 of 592| talnakh
Processing record 436 of 592| atambua
Processing record 437 of 592| maniitsoq
Processing record 438 of 592| kichera
Processing record 439 of 592| buritis
Processing record 440 of 592| vedea
Processing record 441 of 592| zeya
Processing record 442 of 592| ilirska bistrica
Processing record 443 of 592| lompoc
City not found. Skipping...
Processing record 445 of 592| vanavara
Processing record 446 of 592| comodoro rivadavia
City not found. Skipping...
Processing record 448 of 592| port lincoln
Processing record 449 of 592| poian
Processing record 450 of 592| chala
Processing record 451 of 592| point pedro
Processing record 452 of 592| sagaing
Processing record 453 of 592| sao filipe
Proces

In [23]:
#Create dataframe from data 
cities_dict = {"City": city_names, "Cloudiness": cloudiness, "Country": countries, "Date": dates, "Humidity": humidity, 
             "Lat": city_lats, "Lng": city_lngs, "Max Temp": max_temps, "Wind Speed": wind_speed}

cities_df = pd.DataFrame(cities_dict)
cities_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Albany,1,US,1583829902,42,42.60,-73.97,284.26,2.60
1,Castro,61,BR,1583829557,99,-24.79,-50.01,285.52,0.94
2,Camacha,20,PT,1583829624,82,33.08,-16.33,291.15,4.60
3,Bluff,95,NZ,1583829624,91,-46.60,168.33,282.59,1.80
4,Kruisfontein,100,ZA,1583829624,64,-34.00,24.73,295.41,4.26
